In [8]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama3")
llm.invoke("The first man on the moon was ...")

'According to NASA and most reputable sources, the first humans to walk on the Moon were Neil Armstrong and Edwin "Buzz" Aldrin. They landed on July 20, 1969, as part of the Apollo 11 mission.\n\nNeil Armstrong became the first person to set foot on the Moon, famously declaring "That\'s one small step for man, one giant leap for mankind" as he stepped off the lunar module Eagle onto the Moon\'s surface.\n\nBuzz Aldrin joined him shortly afterwards, and the two astronauts spent about two and a half hours exploring the Moon\'s surface, collecting samples, and conducting experiments.'

In [10]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm = Ollama(
    model="llama3", callback_manager=CallbackManager([StreamingStdOutCallbackHandler()])
)
llm.invoke("The first man on the moon was ...")

/var/folders/_9/nxb2f4f13px9by3vcfvrckp40000gn/T/ipykernel_20467/2140520511.py:4: DeprecationWarning: callback_manager is deprecated. Please use callbacks instead.
  llm = Ollama(


Neil Armstrong! On July 20, 1969, Neil Armstrong and Edwin "Buzz" Aldrin became the first humans to set foot on the Moon as part of the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he stepped off the lunar module Eagle onto the Moon's surface.

'Neil Armstrong! On July 20, 1969, Neil Armstrong and Edwin "Buzz" Aldrin became the first humans to set foot on the Moon as part of the Apollo 11 mission. Armstrong famously declared, "That\'s one small step for man, one giant leap for mankind," as he stepped off the lunar module Eagle onto the Moon\'s surface.'

In [16]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [17]:
# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable